<a href="https://colab.research.google.com/github/Timliuw/DP-Tim/blob/main/CI_Tim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time



In [ ]:
#for experiments purpose
import gspread
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import files
# uploaded = files.upload()
import os
os.listdir()

scope = [
    "https://spreadsheets.google.com/feeds",  # Google Sheets API
    "https://www.googleapis.com/auth/drive"   # Google Drive API
]

# Trigger the file upload dialog
try:
    creds = ServiceAccountCredentials.from_json_keyfile_name("sublime-mission-447319-c2-0df1612796a8.json", scope)
except:
   uploaded = files.upload()

creds = ServiceAccountCredentials.from_json_keyfile_name("sublime-mission-447319-c2-0df1612796a8.json", scope)
client = gspread.authorize(creds)

spreadsheet = client.open_by_key("1yVVt5ACdBbjsZ_7IiRkvPGIHmpKe8mFXcvyL0PVP2cE")
worksheet1 = spreadsheet.get_worksheet(0)
worksheet2 = spreadsheet.get_worksheet(1)

data = worksheet1.get_all_records()
print(data)

def get_color(curr_value, prev_value):
    if curr_value < 0.6 * prev_value:
        return {"red": 0.6, "green": 1.0, "blue": 0.6}  # very green
    elif curr_value < prev_value:
        return {"red": 0.8, "green": 1.0, "blue": 0.8}  # green
    elif curr_value > 1.4 * prev_value:
        return {"red": 1.0, "green": 0.6, "blue": 0.6}  # very red
    elif curr_value > prev_value:
        return {"red": 1.0, "green": 0.8, "blue": 0.8}  # red
    return None

def write_to_sheet(worksheet, row_data, compare=False):
    # Append the new row to the sheet
    worksheet.append_row(row_data)

    # Format colour according to the comparing results
    if compare:
        sheet_data = worksheet.get_all_values()
        last_row_index = len(sheet_data)
        if last_row_index > 1:
            last_row = sheet_data[-1]
            prev_row = sheet_data[-2]
            for col_index, (curr_value, prev_value) in enumerate(zip(last_row, prev_row), start=1):
                try:
                    curr_value = float(curr_value)
                    prev_value = float(prev_value)
                    if (worksheet.row_values(1)[col_index-1] == "est median"):
                      prev_value = abs(true_med - prev_value)
                      curr_value = abs(true_med - curr_value)
                    color = get_color(curr_value, prev_value)
                    if color:
                        worksheet.format(f"{chr(64 + col_index)}{last_row_index}", {"backgroundColor": color})
                except ValueError:
                    continue

[{'Method': 'EM_CI', 'Data_id': '7df76a8134e644ba81dbae6f3ea321dd', 'n': 4000, 'domain size': 4000, 'sens': 1, 'beta': 0.01, 'eps': 1, 'num repeat': 100, 'true median': 2038, 'est median': 2046.47, 'correct rate': 1, 'avg CI length': 253.85, 'error quantile': 16, 'avg error': 4.01, 'avg error(std dev)': 3.888888889, 'avg error(IQR)': 2.367088608, 'avg rank error': 2.91, 'relative CI Width': 15.865625, 'avg time': 0.001256325245}, {'Method': 'our_CI', 'Data_id': '7df76a8134e644ba81dbae6f3ea321dd', 'n': 4000, 'domain size': 4000, 'sens': 1, 'beta': 0.01, 'eps': 1, 'num repeat': 100, 'true median': 2038, 'est median': 2035.317186, 'correct rate': 1, 'avg CI length': 220.11, 'error quantile': 16.4902905, 'avg error': 3.381553803, 'avg error(std dev)': 3.249142321, 'avg error(IQR)': 2.826607994, 'avg rank error': 4.37, 'relative CI Width': 13.3478546, 'avg time': 0.1002880907}, {'Method': 'EM_CI', 'Data_id': '7df76a8134e644ba81dbae6f3ea321dd', 'n': 4000, 'domain size': 4000, 'sens': 1, 'bet

In [ ]:
def discretize(D, b):
    n = len(D)
    discreteD = np.zeros(n)
    for i in range(n):
        discreteD[i] = int(D[i]/b)
    return discreteD

def discrete(D):
    n = len(D)
    newD = np.zeros(n)
    previous = -1
    for i in range(n):
        if D[i]!=previous:
            newD[i] = n*D[i]
            #print(newD[i])
            previous = D[i]
        elif D[i]==previous:
            newD[i] = newD[i-1]+1

    return newD

def count(D, a):
    counter = 0
    n = len(D)
    low = 0
    up=n-1
    mid = int((low+up)/2)
    while True:
        if D[mid]>a:
            up=mid
            mid = int((low+up)/2)
        if D[mid]<a:
            low=mid
            mid = int((low+up)/2)
        if D[mid]==a:
            i=0
            while D[mid+i]==a:
                i+=1
            return mid+i

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else:
                low = mid
            mid = (high+low)/2

        return high
    if y==1/2:
        return 0

def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

In [ ]:
def constructu(eps, a,b, D):
    global u
    global l
    global weight
    #[a,b] denotes range
    n = len(D)
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+2)#score changing point
    l[0] = a
    for i in range(n+1):
        if i<=int(n/2):
            u[i] = -int(n/2)-1+i
            l[i+1]= D[i]
        #u[int(n/2+1)] = 0
        #l[int(n/2+2)] = D[int(n/2)]
        if i>int(n/2):
            u[i] = int(n/2)+1-i
            l[i]= D[i-1]

    l[n+1] = b
    u[n+1] = -n-1-u[0]
    weight = []
    for i in range(int(n/2)+1):
        weight.append((l[i+1]-l[i])*math.pow(np.e, eps*u[i]/2))
    weight.append(1)
    for i in range(int(n/2)+1,n+1):
        weight.append((l[i]-l[i-1])*math.pow(np.e, eps*u[i]/2))

    totalWeight = sum(weight)
    weight = weight/totalWeight
    #print(len(weight))

def EMMedian_new():
    i = np.random.choice(list(range(len(l))), p=weight)
    if i==int(n/2)+1:
        return int(l[i]/n)
    if i<int(n/2)+1:
        return int(np.random.randint(l[i], l[i+1],dtype=np.int64)/n)
    if i>int(n/2)+1:
        return int(np.random.randint(l[i-1], l[i],dtype=np.int64)/n)

def constructu_CI(eps, beta, N):
    global u1
    global weight1
    global u2
    global weight2
    global factor

    factor = int(8/eps*np.log(4*n*N/beta))
    print("factor is "+str(factor))
    u1 = np.zeros(n+2)
    u2 = np.zeros(n+2)
    for i in range(n+2):
        if i<=int(n/2)+1:
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)

    idx = int(n/2)+1-factor
    weight1 = []
    for i in range(idx):
        weight1.append((l[i+1]-l[i])*math.pow(np.e, eps*u1[i]/4))
    weight1.append(1)
    for i in range(idx,n+1):
        weight1.append((l[i]-l[i-1])*math.pow(np.e, eps*u1[i]/4))

    totalWeight1 = sum(weight1)
    weight1 = weight1/totalWeight1

    idx = int(n/2)+1+factor
    weight2 = []
    for i in range(idx):
        weight2.append((l[i+1]-l[i])*math.pow(np.e, eps*u2[i]/4))
    weight2.append(1)
    for i in range(idx,n+1):
        weight2.append((l[i]-l[i-1])*math.pow(np.e, eps*u2[i]/4))

    totalWeight2 = sum(weight2)
    weight2 = weight2/totalWeight2

def EMMedianCI():
    i1 = np.random.choice(list(range(len(l))), p=weight1)
    if i1==int(n/2)+1-factor:
        x1= int(l[i1]/n)
    if i1<int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1], l[i1+1],dtype=np.int64)/n)
    if i1>int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1-1], l[i1],dtype=np.int64)/n)

    i2 = np.random.choice(list(range(len(l))), p=weight2)
    if i2==int(n/2)+1+factor:
        x2= int(l[i2]/n)
    if i2<int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2], l[i2+1],dtype=np.int64)/n)
    if i2>int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2-1], l[i2],dtype=np.int64)/n)

    est = EMMedian_new()
    indicator=0
    if D[int(n/2)]<=x2 and D[int(n/2)]>=x1:
        indicator=1

    return x1,x2 ,indicator,est

In [ ]:
def SVTquantile(eps, beta, D, tau, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    T = tau

    for i in range(rad+1):
        Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i
def SVTmedian(eps, beta, D, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    n=len(D)
    T = int(n/2)

    for i in range(rad+1):
        Ti = T+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i

def SVTMedian(eps, beta, D):
    n = len(D)
    rad = 1000000
    Q = SVTmedian(eps, beta, D, rad)

    factor = int(16/eps*math.log(2*(2*rad+1)**2 *math.pi**2/(3*beta))+8/eps*math.log(4/beta))+1
    ##Then run SVT, do not double
    ##find the n/2 +- factor quantile
    T1 = int(n/2)-1
    T2 = int(n/2)+factor
    XLeft = SVTquantile(eps/2, beta/2, D, T1, rad)
    #print(XLeft)
    XRight = SVTquantile(eps/2, beta/2, D, T2, rad)
    #print(XRight)
    indicator=0
    if D[int(n/2)]<=Q+ XRight and D[int(n/2)]>=XLeft:
        indicator=1

    return Q, XLeft,XRight,indicator

In [ ]:
def smoothMedian(eps, beta, D,delta=0):
    #given D, first compute S(D)
    n = len(D)
    m = int(n/2)
    rad = 1000000
    SS = 0
    if delta==0:
        alpha = eps/20
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))

        Q = D[int(n/2)]+20*SS/eps*CauchyNoise()
        #print(Q)
        indicator=0
        factor = 20*noisySS/eps*inver_F(1-beta/4)
        if D[int(n/2)]<=Q+ factor and D[int(n/2)]>=Q-factor:
            indicator=1

        return Q, factor,indicator
    else:
        alpha = eps/(4*math.log(1/delta))
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print(noisySS)
        Q = D[int(n/2)]+4*SS/eps*LapNoise()


        indicator=0
        if D[int(n/2)]<=Q+ 4*noisySS/eps*math.log(2/beta) and D[int(n/2)]>=Q-4*noisySS/eps*math.log(2/beta):
            indicator=1

        return Q, 4*noisySS/eps*math.log(2/beta),indicator


In [ ]:
# Parameters
n = 4000  # 45#000
domain_size = 4000  # 40#000
beta = 0.01
sens = 1
eps = 1
num_repeat=100

# Parameters for our_CI
beta_1 = beta / 2
beta_2 = beta_1
eps_1 = 0.5 * eps
eps_2 = eps - eps_1

D = np.random.uniform(0, domain_size, n)
D.sort()
D = discretize(D, 1)
newD = discrete(D)
true_med = np.median(D)  # GET EXACT MEDIAN
b_list = range(0, domain_size, 1)  # [0,10,20,30,40,50,60,70,80,90,100]
m = len(b_list)

print(true_med)

2015.5


In [ ]:
# Our Additions (September 27th)
# Exponetial Mechanism + Confidence Intervals

#ci = (2 * sens / eps) * (np.log(domain_size / beta))
#print(ci)

#np.unique(D,return_counts=True)

#vals, counts = np.unique(D,return_counts=True)
#print(vals)
#vals = [0.0] + vals.tolist()
#print(vals)

def basic_EM(eps,beta,D,domain_size):
    print("Results for EM, beta = "+str(beta) + ", epsilon = " + str(eps))
    #print(D)
    #D = D.sort() # D is sorted
    interval_prob = []
    vals, counts = np.unique(D,return_counts=True)
    #print(vals)
    #print(counts)
    rank = 0
    vals = [0.0] + vals.tolist()
    cdf = 0.0
    for i in range(1,len(vals)):
        rank += counts[i-1]
        utility = -1.0* np.abs(rank - (len(D)/2))
        #print(rank)
        #print(utility)
        #print(math.pow(np.e, eps*utility))
        p = (vals[i] - vals[i-1])* math.pow(np.e, eps*utility)
        cdf += p
        interval_prob.append(cdf)
    rand_val = cdf * np.random.uniform(0,1)
    #print(cdf,rand_val)
    for i in range(1,len(vals)):
        if rand_val <= interval_prob[i]:
            sens = 1
            ci = (2 * sens / eps) * (np.log(domain_size / beta))
            med_val = np.random.uniform(vals[i-1],vals[i])

            print("median rank: %d out of %d" %(i,len(vals)) )
            print("range for median rank: (%f,%f)" %(vals[i-1],vals[i]))
            #print("sampled median value: (%f)" % (med_val)) #need to convert from rank to true value
            print("size of confidence interval: %f" %(ci)) # for rank
            break

basic_EM(eps,beta,D,domain_size)

Results for EM, beta = 0.01, epsilon = 1
median rank: 1265 out of 2525
range for median rank: (2013.000000,2014.000000)
size of confidence interval: 25.798440


In [ ]:
# November 22nd / November 29th

#1) Learn noisy median o of distribution using Exponential Mechanism
def EM_median(eps,beta,D,domain_size):
    #print("Results for EM, beta = "+str(beta) + ", epsilon = " + str(eps))
    interval_prob = []
    vals, counts = np.unique(D,return_counts=True)
    rank = 0
    vals = [0.0] + vals.tolist()
    cdf = 0.0
    rank_dict = {}
    for i in range(1,len(vals)):
        rank += counts[i-1]
        rank_dict[vals[i]] = rank
        utility = -1.0* np.abs(rank - (len(D)/2))
        p = (vals[i] - vals[i-1])* math.pow(np.e, eps*utility)
        cdf += p
        interval_prob.append(cdf)
    rand_val = cdf * np.random.uniform(0,1)
    rank_o = 0
    for i in range(1,len(vals)):
        if rand_val <= interval_prob[i]:
            sens = 1
            ci = (2 * sens / eps) * (np.log(domain_size / beta))
            o = np.random.uniform(vals[i-1],vals[i])
            rank_o = rank_dict[vals[i-1]]
            #print("median rank: %d out of %d" %(rank_dict[vals[i-1]],len(vals)) )
            #print("range for median rank: (%f,%f)" %(vals[i-1],vals[i]))
            #print("sampled median value: (%f)" % (o)) #need to convert from rank to true value
            #print("size of confidence interval: %f" %(ci)) # for rank
            break
    return o,rank_o,rank_dict

#2)
def find_rank(rank_dict,val):
    ranks = list(rank_dict.values())
    keys = list(rank_dict.keys())
    output = 0
    for i in range(len(ranks)-1):
        if val < keys[i+1]:
            output = ranks[i]
            break
    #print(rank_dict[output])
    return output#int(output)


def SVT_median(o,rank_o,rank_dict,eps,beta,D,domain_size,T,b_list):
    #print("T: ", T)
    noisy_T = T + np.random.laplace(2/eps)
    #print("noisy_T: ", noisy_T)
    #print("rank_dict", list(rank_dict.values()))
    #print("rank_keys", list(rank_dict.keys()))

    sensitivity = 1
    #b_max = 10
    #step = 1
    bounds = domain_size
    for b in b_list: #range(0,b_max,step):
        perturbed_value = min(abs(find_rank(rank_dict, o+b)-rank_o),abs(find_rank(rank_dict, o-b)-rank_o)) + np.random.laplace(2*sensitivity/eps)
        if perturbed_value > noisy_T:
            bounds = b
            break
    #print(max(o-bounds,0))
    #print(min(o+bounds,domain_size-1))
    return max(o-bounds,0),min(o+bounds,domain_size-1)

##### main function ######

def our_CI():
    C = (2/eps_1) *(np.log(domain_size/beta_1))
    alpha = (8*np.log(m)+np.log(2/beta_2)) / eps_2
    #print("C", C)
    #print("alpha", alpha)
    T = C + alpha
    o,rank_o,rank_dict = EM_median(eps_1,beta_1,D,domain_size)
    lower,upper = SVT_median(o,rank_o,rank_dict,eps_2,beta_1,D,domain_size,T,b_list)
    #print("median:",o)
    #print("The bounds are:")
    #print('[{},{}]'.format(lower,upper))
    indi = -1 #if CI contains true med
    if true_med < upper and true_med > lower:
        indi = 1
    else:
        indi = 0
    return lower,upper,indi,o

our_CI()

(1820.8303668417918, 2206.830366841792, 1, 2013.8303668417918)

In [ ]:
#Test for our_CI
def test_ours():
  len_eps = []
  err_eps = []
  est_median = []
  times = []
  rank_error = []
  for i in range(len(beta)):
      print("Results for EM based median, beta = "+str(beta[i]))
      errors = []
      lengths = []
      correct_count = 0

      for j in range(num_repeat):
          start = time.time()
          xl,xr,indi,est = our_CI()#EMMedianCI()
          end = time.time()
          times.append(end-start)
          correct_count+=indi
          errors.append(abs(est-true_med))
          rank_error.append(abs(n/2 - find_rank(rank_dict, est))) #new
          lengths.append((xr-xl)/(2))
          est_median.append((xr+xl)/2)

      correct_rate = correct_count/num_repeat
      avgLength = sum(lengths)/num_repeat
      errors.sort()
      errorQuantile = errors[int(num_repeat*(1-beta[i]))]
      avg_error = np.average(errors)
      avgTime = sum(times)/len(times)

      # new
      # error_avg
      errors = np.array(errors)
      avg_error = np.average(errors)
      std_dev = np.std(errors)
      stddev_based_avg_err = np.average(errors[np.abs(errors - avg_error) < 3 * std_dev])
      q1 = np.percentile(errors, 25)
      q3 = np.percentile(errors, 75)
      iqr = q3 - q1
      lower_bound = q1 - 1.5 * iqr
      upper_bound = q3 + 1.5 * iqr
      IQR_based_avg_err = np.average(errors[(errors > lower_bound) & (errors < upper_bound)])
      #rank_avg
      avg_rank_error = np.average(rank_error)

      print("estimated median value: "+ str(np.average(est_median)))
      print("correct rate = ", correct_rate)
      print("Average CI length = "+str(avgLength))
      print("error quantile is "+ str(errorQuantile))
      print("CI length/ error quantile = "+str(avgLength/errorQuantile))
      print("Average Time to find CI = " + str(avgTime))

      row_data = [
          "our_CI",
          data_uuid,
          n,
          domain_size,
          sens,
          beta[i],
          eps[i],
          num_repeat,
          true_med,
          np.average(est_median),
          correct_rate,
          avgLength,
          errorQuantile,
          avg_error,
          stddev_based_avg_err,
          IQR_based_avg_err,
          avg_rank_error,
          avgLength / errorQuantile,
          avgTime
      ]
      write_to_sheet(worksheet1, row_data, compare = True)

      len_eps.append(avgLength)
      err_eps.append(errorQuantile)
  print(len_eps)
  print(err_eps)

In [ ]:
#Test for their algorithm
def test_theirs():
  len_eps = []
  err_eps = []
  est_median = []
  times = []
  rank_error = []
  for i in range(len(beta)):
      print("Results for EM based median, beta = "+str(beta[i]))
      errors = []
      lengths = []
      correct_count = 0
      n = len(newD)
      rad = 10000000
      left=-rad
      constructu(eps[i], left*n,rad*n, newD)

      constructu_CI(eps[i],beta[i], rad)
      for j in range(num_repeat):
          start = time.time()
          xl,xr,indi,est = EMMedianCI()
          end = time.time()
          times.append(end-start)
          correct_count+=indi
          errors.append(abs(est - true_med))
          rank_error.append(abs(n/2 - find_rank(rank_dict, est))) #new
          lengths.append((xr-xl)/(2))
          est_median.append((xr+xl)/2)


      correct_rate = correct_count/num_repeat
      avgLength = sum(lengths)/num_repeat
      errors.sort()
      errorQuantile = errors[int(num_repeat*(1-beta[i]))]
      avgTime = sum(times)/len(times)
      # new
      # error_avg
      errors = np.array(errors)
      avg_error = np.average(errors)
      std_dev = np.std(errors)
      stddev_based_avg_err = np.average(errors[np.abs(errors - avg_error) < 3 * std_dev])
      q1 = np.percentile(errors, 25)
      q3 = np.percentile(errors, 75)
      iqr = q3 - q1
      lower_bound = q1 - 1.5 * iqr
      upper_bound = q3 + 1.5 * iqr
      IQR_based_avg_err = np.average(errors[(errors > lower_bound) & (errors < upper_bound)])
      #rank_avg
      avg_rank_error = np.average(rank_error)

      print("estimated median value: "+ str(np.average(est_median)))
      print("correct rate = " +str(correct_count/num_repeat))
      print("Average CI length = "+str(avgLength))
      print("error quantile is "+ str(errorQuantile))
      print("CI length/ error quantile = "+str(avgLength/errorQuantile))
      print("Average Time to find CI = " + str(sum(times)/len(times)))
      row_data = [
          "EM_CI",
          data_uuid,
          n,
          domain_size,
          sens,
          beta[i],
          eps[i],
          num_repeat,
          true_med,
          np.average(est_median),
          correct_rate,
          avgLength,
          errorQuantile,
          avg_error,
          stddev_based_avg_err,
          IQR_based_avg_err,
          avg_rank_error,
          avgLength / errorQuantile,
          avgTime
      ]
      write_to_sheet(worksheet1, row_data)

      len_eps.append(avgLength)
      err_eps.append(errorQuantile)
  print(len_eps)
  print(err_eps)


In [ ]:
# Data Parameters
n = 4000  # 45#000
domain_size = 4000  # 40#000
D = np.random.uniform(0, domain_size, n)
D.sort()
D = discretize(D, 1)
newD = discrete(D)
true_med = np.median(D)  # GET EXACT MEDIAN
print("true median: ", true_med)

#For rank_dict
vals, counts = np.unique(D,return_counts=True)
rank_dict = {}
rank = 0
vals = [0.0] + vals.tolist()
for i in range(1,len(vals)):
    rank += counts[i-1]
    rank_dict[vals[i]] = rank
#For Data Persistence
import pandas as pd
import uuid
df = pd.DataFrame(D)
data_uuid = uuid.uuid4().hex
file_name = f"data_{data_uuid}.csv"
df.to_csv(file_name, index=False, header=False)
files.download(file_name)
print(f"File saved as {file_name}")



true median:  2027.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File saved as data_2795f6b9bde84262a61a2f101bdadf61.csv


In [ ]:
# CI Parameters
eps = [1]
beta = [0.01]
sens = 1 #fixed
num_repeat=100


# for val, rank in rank_dict.items():
#     print(val,"  ",rank)

# Parameters for our_CI
beta_1 = beta[0] / 2
beta_2 = beta_1
eps_1 = 0.5 * eps[0]
eps_2 = eps[0] - eps_1
b_list = range(0, domain_size, 1)  # [0,10,20,30,40,50,60,70,80,90,100]
m = len(b_list)



In [ ]:
#Test both
test_theirs()
print("-------------------First Test Done--------------------")
test_ours()
print("-------------------Second Test Done--------------------")

Results for EM based median, beta = 0.01
factor is 243
estimated median value: 2033.13
correct rate = 1.0
Average CI length = 261.72
error quantile is 20.0
CI length/ error quantile = 13.086000000000002
Average Time to find CI = 0.0013172793388366699
[261.72]
[20.0]
-------------------First Test Done--------------------
Results for EM based median, beta = 0.01
estimated median value: 2022.9027765812714
correct rate =  1.0
Average CI length = 219.88
error quantile is 22.957947516568083
CI length/ error quantile = 9.577511223131728
Average Time to find CI = 0.11392672538757324
[219.88]
[22.957947516568083]
-------------------Second Test Done--------------------
